# Đồ án cuối kỳ

**Các thành viên:**
1. MSSV: 18120143 - Họ tên: Nguyễn Đình Hoàng Phúc
2. MSSV: 18120512 - Họ tên: Lê Đặng Thiên Phúc

---

## Import thư viện

In [230]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Thu thập dữ liệu

Trong đồ án lần này nhóm chúng em sử dụng bộ dữ liệu thông tin các **Ứng dụng trên Google Play Store**. <br>
Bộ dữ liệu được tạo vào 05/09/2018 và cập nhật lần cuối vào 03/02/2019 trên trang Kaggle, [đường dẫn](https://www.kaggle.com/lava18/google-play-store-apps). Bộ dữ liệu được thu thập bằng cách crawl từ trang web của Google Play và có chứa nhiều thông tin liên quan đến ứng dụng như tên, kích thước, rating,... <br>
Bộ dữ liệu gồm 2 bảng là "googleplaystore.csv" chứa thông tin chi tiết về các ứng dụng và "googleplaystore_user_reviews.csv" chứa thông tin về  top 100 bình luận liên quan nhất của từng ứng dụng.

## Khám phá dữ liệu

### Đọc dữ liệu từ file .csv vào Dataframe

In [231]:
gps_df = pd.read_csv('googleplaystore.csv')
gps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


### Dữ liệu gồm bao nhiêu dòng và cột?

In [232]:
num_rows, num_cols = gps_df.shape
print('Bộ dữ liệu có {} dòng và {} cột'.format(num_rows, num_cols))

Bộ dữ liệu có 10841 dòng và 13 cột


### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng là chi tiết thông tin của một ứng dụng trên Google Play Store. Có vẻ không có dòng nào có ý nghĩa khác.

### Dữ liệu có các dòng bị lặp không?

In [233]:
duplicated_rows = gps_df.duplicated()
num_duplicated_rows = len(gps_df[duplicated_rows])
print("Có {} dòng bị lặp.".format(num_duplicated_rows))

Có 483 dòng bị lặp.


#### Tiền xử lí:
Ta cần xóa các dòng bị lặp đi để các bước tính toán sau được chính xác.

In [234]:
gps_df = gps_df.drop_duplicates()
num_rows, num_cols = gps_df.shape
print('Bộ dữ liệu sau khi xóa dòng lặp có {} dòng và {} cột'.format(num_rows, num_cols))

Bộ dữ liệu sau khi xóa dòng lặp có 10358 dòng và 13 cột


### Mỗi cột có ý nghĩa gì?

Dưới đây là phần mô tả của tác giả về các cột trong file "googleplaystore.csv":
- **App**: Application name
- **Category**: Category the app belongs to
- **Rating** : Overall user rating of the app (as when scraped)
- **Reviews**: Number of user reviews for the app (as when scraped)
- **Size**: Size of the app (as when scraped)
- **Installs**: Number of user downloads/installs for the app (as when scraped)
- **Type**: Paid or Free
- **Price**: Price of the app (as when scraped)
- **Content Rating**: Age group the app is targeted at - Children / Mature 21+ / Adult
- **Genres**: An app can belong to multiple genres (apart from its main category). For eg, a musical family game will belong to Music, Game, Family genres.
- **Last Updated**: Date when the app was last updated on Play Store (as when scraped)
- **Current Ver**: Current version of the app available on Play Store (as when scraped)
- **Android Ver**: Min required Android version (as when scraped)

### Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp không?

In [235]:
col_dtypes = gps_df.dtypes
col_dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

Chỉ có cột `Rating` có kiểu `float`, các cột còn lại đều có kiểu là `object`. Do không có cảnh báo về mixed typed. Dự đoán các ngoại trừ `Rating` đều có kiểu dữ liệu là string. Nếu dựa vào kiểu dữ liệu, chỉ có cột `Rating` là có kiểu numerical còn các cột còn lại đều là `categorical`. Tuy nhiên dựa vào ý nghĩa của các cột thì không phải như vậy. Thử quan sát trước các cột có kiểu dữ liệu là `object` để nhận xét và xử lý thêm:

Các hàm hỗ trợ:

In [236]:
def missing_ratio(s):
    return s.isna().mean() * 100
def num_diff_vals(s):
    return s.nunique()
def diff_vals(s):
    return s.dropna().unique()

Quan sát các cột có kiểu dữ liệu `object`.

In [237]:
categorical_col_df = gps_df[['App', 'Category', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated','Current Ver', 'Android Ver'
]]
categorical_col_profiles_df = categorical_col_df.agg([missing_ratio, num_diff_vals, diff_vals])
categorical_col_profiles_df

,App,Category,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.009654,0.0,0.009654,0.0,0.0,0.077235,0.028963
num_diff_vals,9660,34,6002,462,22,3,93,6,120,1378,2832,33
diff_vals,[Photo Editor & Candy Camera & Grid & ScrapBoo...,"[ART_AND_DESIGN, AUTO_AND_VEHICLES, BEAUTY, BO...","[159, 967, 87510, 215644, 167, 178, 36815, 137...","[19M, 14M, 8.7M, 25M, 2.8M, 5.6M, 29M, 33M, 3....","[10,000+, 500,000+, 5,000,000+, 50,000,000+, 1...","[Free, Paid, 0]","[0, $4.99, $3.99, $6.99, $1.49, $2.99, $7.99, ...","[Everyone, Teen, Everyone 10+, Mature 17+, Adu...","[Art & Design, Art & Design;Pretend Play, Art ...","[January 7, 2018, January 15, 2018, August 1, ...","[1.0.0, 2.0.0, 1.2.4, Varies with device, 1.1,...","[4.0.3 and up, 4.2 and up, 4.4 and up, 2.3 and..."


Quan sát kỹ hơn hai cột `Current Ver` và `Android Ver`

In [238]:
categorical_col_profiles_df['Current Ver']['diff_vals']

array(['1.0.0', '2.0.0', '1.2.4', ..., '1.0.612928', '0.3.4', '2.0.148.0'],
      dtype=object)

In [239]:
categorical_col_profiles_df['Android Ver']['diff_vals']

array(['4.0.3 and up', '4.2 and up', '4.4 and up', '2.3 and up',
       '3.0 and up', '4.1 and up', '4.0 and up', '2.3.3 and up',
       'Varies with device', '2.2 and up', '5.0 and up', '6.0 and up',
       '1.6 and up', '1.5 and up', '2.1 and up', '7.0 and up',
       '5.1 and up', '4.3 and up', '4.0.3 - 7.1.1', '2.0 and up',
       '3.2 and up', '4.4W and up', '7.1 and up', '7.0 - 7.1.1',
       '8.0 and up', '5.0 - 8.0', '3.1 and up', '2.0.1 and up',
       '4.1 - 7.1.1', '5.0 - 6.0', '1.0 and up', '2.2 - 7.1.1',
       '5.0 - 7.1.1'], dtype=object)

Quan sát dữ liệu và dựa và ý nghĩa, ta thấy:
- Các cột `App`, `Category`,`Type`, `Content Rating`, `Genres` chắc chắn được xếp vào categorical. 
- Các cột `Reviews`: số lượng người đánh giá, `Size`: kích thước ứng dụng, `Installs`: số lượt tải, `Price`: giá tiền mua, `Last Update`: lần cập nhật cuối nên xét vào dạng numerical. 
- Cột `Current Version` khá khó phân loại do việc đánh số phiên bản do nhà sản xuất ứng dụng phụ trách chỉ có giá trị so sánh nội trong ứng dụng, không thể so sánh con số này giữa các ứng dụng với nhau cũng không thể xét nhà sản xuất có thường cập nhật hay không dựa vào con số này, do đó tạm thời ta chưa xử lý cột này. 
- Cột `Android Ver` là dữ liệu dạng khoảng, có thể tiền xử lý chia thành hai cột phiên bản hệ điều hành thấp nhất và cao nhất.

Ta tiền xử lý chuyển dữ liệu các cột được phân loại là numerical có kiểu object về lại kiểu số.

#### Tiền xử lý

##### Chuyển cột `Reviews` sang kiểu số.
Quan sát cột Reviews thấy đa phần đều có kiểu số nhưng mặc định lại không đọc thành kiểu số được. Ta kiểm tra xem có bao nhiêu dòng gây ra việc này?

In [240]:
pd.to_numeric(gps_df['Reviews'],errors = 'coerce').isna().sum()

1

Do chỉ có một dòng nên chấp nhận để giá trị `N/A` ở Reviews cho dòng này. Giá trị của dòng gây lỗi này cũng là `3.0M`, có thể crawl bị lỗi sang cột số lượt tải.

Chuyển dữ liệu cột `Reviews` sang dạng số:

In [241]:
gps_df['Reviews'] = pd.to_numeric(gps_df['Reviews'],errors = 'coerce')

##### Chuyển cột `Size` sang dạng số:
Ta chỉ chấp nhận các pattern string của cột `Size` có dạng là: Varies with device, kích thước dưới dạng MB và kích thước dưới dạng KB. 
Đếm xem thử có bao nhiêu dòng không có dạng như vậy:

In [242]:
out_of_pattern_rows = gps_df['Size'].str.extract(r'((?:Varies with device)|(?:\d+.\d[M])|(?:\d+[M])|(?:\d[k]))', expand=False).isna()
out_of_pattern_rows.sum()

1

Chỉ có 1 dòng, cũng không đáng kể.

Mặc dù ý nghĩa của 'Varies with device' và không chuyển được về kiểu số là khác nhau. Trong đồ án này nhóm chúng em xem hai giá trị này là tương đương nhau là không thu thập được và mang giá trị N/A.

In [243]:
gps_df.loc[out_of_pattern_rows,'Size'] = np.nan

In [244]:
gps_df.loc[gps_df['Size']=='Varies with device','Size'] = np.nan

Dùng regrex để tách số.

In [222]:
size_nums = gps_df['Size'].str.extract(r'((?:\d+.\d)|(?:\d+)|(?:\d))', expand=False).astype('float64')

0        19.0
1        14.0
2         8.7
3        25.0
4         2.8
         ... 
10836    53.0
10837     3.6
10838     9.5
10839     NaN
10840    19.0
Name: Size, Length: 10358, dtype: float64

Các dòng có đơn vị là KB cần được chuyển đổi về cùng đơn vị MB.

In [224]:
size_nums[gps_df['Size'].str[-1]=='k'] *= 0.001

Gán lại vào DataFrame chính.

In [225]:
gps_df['Size'] = size_nums

##### Chuyển cột `Install` sang dạng số: 
Ta chỉ chấp nhận các pattern string của cột `Install` có dạng là: số nguyên và dấu cộng "+" liền sau đó. (1000+, 10000+,...)

In [200]:
numerical_col_df = gps_df[['Rating','Size','Reviews']]
numerical_col_profiles_df = numerical_col_df.agg([missing_ratio, pd.Series.min, pd.Series.max])
numerical_col_profiles_df

,Rating,Size,Reviews
missing_ratio,14.151945,14.750459,1.930688e-02
min,1.000000,0.008500,0.000000e+00
max,19.000000,100.000000,7.815831e+07


## Tiền xử lý

## Đưa ra các câu hỏi có ý nghĩa cần trả lời

## Phân tích dữ liệu để trả lời các câu hỏi trên

## Nhìn lại quá trình làm đồ án

### Thành viên: Nguyễn Đình Hoàng Phúc

### Thành viên: Lê Đặng Thiên Phúc

## Tài liệu tham khảo